# 재난문자방송 발령현황 API Service Crawling

In [2]:
import requests
import json
from tqdm import tqdm
import pandas as pd

In [3]:
# 데이터 확인

url = 'http://apis.data.go.kr/1741000/DisasterMsg2/getDisasterMsgList?serviceKey=JEKSrW3YsUbYoCkbk3oWnYQ4dZmZo9nV71BUEkNkIxIpU6yiMN3xIqMMHyVE9075tTDYa%2Bk3GDOxam5oSmRRmg%3D%3D&pageNo=1&numOfRows=10&type=json&flag=Y&'
req = requests.get(url)
json = req.json()
json

{'DisasterMsg': [{'head': [{'totalCount': 22200},
    {'numOfRows': '10', 'pageNo': '1', 'type': 'JSON'},
    {'RESULT': {'resultCode': 'INFO-0', 'resultMsg': '정상'}}]},
  {'row': [{'create_date': '2020/08/07 14:06:42',
     'location_id': '172',
     'location_name': '인천광역시 남동구',
     'md101_sn': '42708',
     'msg': '[남동구청] 코로나19 50번째 확진자 1명(간석4동) 발생. 방역완료. 세부 이동동선 등은 홈페이지 및 블로그에 게시. reurl.kr/32796BA2YT',
     'send_platform': 'cbs'},
    {'create_date': '2020/08/07 14:05:12',
     'location_id': '215',
     'location_name': '전라북도 정읍시',
     'md101_sn': '42707',
     'msg': '[정읍시청] 집중호우 위험, 개울가, 계곡, 하천변, 절개지 주변 펜션, 가건물, 노후 저수지 하류지역 등 위험지역에서는 대피 등 안전에 유의바랍니다.',
     'send_platform': 'cbs'},
    {'create_date': '2020/08/07 14:00:13',
     'location_id': '203',
     'location_name': '전라북도 고창군',
     'md101_sn': '42706',
     'msg': '[고창군청] 호우주의보 발효 중, 집중호우 위험. 저지대, 침수 우려지역 등 위험지역에서는 가족, 이웃과 정보를 공유하고 안전한 곳으로 대피하시기 바랍니다',
     'send_platform': 'cbs'},
    {'create_date': '2020/08/07 13:59:

In [4]:
# Crawling

def api_crawling():
    url = 'http://apis.data.go.kr/1741000/DisasterMsg2/getDisasterMsgList?serviceKey=JEKSrW3YsUbYoCkbk3oWnYQ4dZmZo9nV71BUEkNkIxIpU6yiMN3xIqMMHyVE9075tTDYa%2Bk3GDOxam5oSmRRmg%3D%3D&pageNo=1&numOfRows=1000&type=json&flag=Y&'
    
    req = requests.get(url)
    
    totalCount = req.json()['DisasterMsg'][0]['head'][0]['totalCount']
    
    create_date = []
    location_id = []
    location_name = []
    md101_sn = []
    msg = []
    send_platform = []
    
    # totalCount가 numOfRows=1000으로 나누어떨어지는 경우
    if (totalCount % 1000) == 0:
        total_page = (totalCount // 1000) + 1
    # totalCount가 numOfRows=1000으로 나누어떨어지지 않는 경우
    else:
        total_page = (totalCount // 1000) + 2
        
        for pageNo in tqdm(range(1, total_page)):
            url = 'http://apis.data.go.kr/1741000/DisasterMsg2/getDisasterMsgList?serviceKey=JEKSrW3YsUbYoCkbk3oWnYQ4dZmZo9nV71BUEkNkIxIpU6yiMN3xIqMMHyVE9075tTDYa%2Bk3GDOxam5oSmRRmg%3D%3D&numOfRows=1000&type=json&flag=Y&pageNo=' + str(pageNo)
            
            req = requests.get(url)
            
            json_data = req.json()
            
            row = json_data['DisasterMsg'][1]['row']
            
            for data in row:
                create_date.append(data['create_date'])
                location_id.append(data['location_id'])
                location_name.append(data['location_name'])
                md101_sn.append(data['md101_sn'])
                msg.append(data['msg'])
                send_platform.append(data['send_platform'])

    msg_api_df = pd.DataFrame({"create_date": create_date, "location_id": location_id, "location_name":location_name,
                               "md101_sn":md101_sn, "msg":msg, "send_platform":send_platform})

    # DataFrame 형식으로 반환
    return msg_api_df

In [5]:
msg_api_df = api_crawling()

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:16<00:00,  1.37it/s]


In [6]:
msg_api_df

,create_date,location_id,location_name,md101_sn,msg,send_platform
0,2020/08/07 14:06:42,172,인천광역시 남동구,42708,[남동구청] 코로나19 50번째 확진자 1명(간석4동) 발생. 방역완료. 세부 이동...,cbs
1,2020/08/07 14:05:12,215,전라북도 정읍시,42707,"[정읍시청] 집중호우 위험, 개울가, 계곡, 하천변, 절개지 주변 펜션, 가건물, ...",cbs
2,2020/08/07 14:00:13,203,전라북도 고창군,42706,"[고창군청] 호우주의보 발효 중, 집중호우 위험. 저지대, 침수 우려지역 등 위험지...",cbs
3,2020/08/07 13:59:35,216,전라북도 진안군,42705,"[진안군청]현재 진안군강수량 130mm이며, 앞으로도 많은비가 내릴 예상입니다. ...",cbs
4,2020/08/07 13:58:51,140,서울특별시 강서구,42704,[강서구청] 강서구 확진자 1명 발생(해외입국자). 상세내용은 블로그 참조바랍니다....,cbs
...,...,...,...,...,...,...
22195,2011/11/30 14:08:13,"3,4,5,6,7,9,91,92,97,163,164,166","강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,강원도 양양...",25,"[국민안전처]\n30일14시동해중부먼바다,15시 동해남부전해상 풍랑경보, 어선은 출...",cbs
22196,2011/11/30 12:38:47,"3,17,18","강원도 강릉시,강원도 평창군,강원도 홍천군",24,"[국민안전처]\n30일12:30 강릉,평창,홍천산간지역 대설경보,대중교통이용,비닐하...",cbs
22197,2011/11/30 07:40:31,"4,7,9,12","강원도 고성군,강원도 속초시,강원도 양양군,강원도 인제군",23,[국민안전처]\n30일07:30 강원산\n간(속초 고성 양\n양 인제) 대설경\n보...,cbs
22198,2011/11/23 19:05:24,"187,188,191,193,194,195,198,199,200,203,204,20...","전라남도 목포시,전라남도 무안군,전라남도 신안군,전라남도 영광군,전라남도 영암군,전...",12,"[국민안전처]\n23일 20시 제주남쪽먼바다, 서해남부먼바다 풍랑경보, 어선출항 금...",cbs


In [6]:
# send_platform'열의 중복 여부 확인
v = msg_api_df.duplicated('send_platform')

In [7]:
list = []
for i in range(len(msg_api_df)):
    if v[i] == False:
        list.append(i)

In [8]:
msg_api_df.loc[list]

,create_date,location_id,location_name,md101_sn,msg,send_platform
0,2020/08/06 14:20:00,49,경기도 평택시,42609,"[평택시청]관내 하천 상류지역에 내린 집중호우로 위험 할 수 있으니 하천변, 하천둔...",cbs
7552,2020/05/11 19:45:42,"2,21,136,168","강원도 전체,경기도 전체,서울특별시 전체,인천광역시 전체",35056,[기상청] 05월11일19:45 북한 강원 평강 북북서쪽 37km 지역 규모4.0 ...,cbs_kma
20216,2017/01/27 09:50:04,3100000000,울산광역시,2336,"〔울산광역재난안전대책본부〕안전안내, 설날 당일 옥동공원묘지 일대 교통혼잡 및 교통사...",SafeOn


In [9]:
msg_api_df[msg_api_df["send_platform"] == 'cbs_kma']

,create_date,location_id,location_name,md101_sn,msg,send_platform
7552,2020/05/11 19:45:42,"2,21,136,168","강원도 전체,경기도 전체,서울특별시 전체,인천광역시 전체",35056,[기상청] 05월11일19:45 북한 강원 평강 북북서쪽 37km 지역 규모4.0 ...,cbs_kma
8178,2020/05/03 22:10:42,"98,179","광주광역시 전체,전라남도 전체",34429,[기상청] 05월03일22:07 전남 해남군 서북서쪽 21km 지역 규모3.1 지진...,cbs_kma
17528,2020/01/30 00:56:43,"74,104,238","경상북도 전체,대구광역시 전체,충청북도 전체",25075,[기상청] 01월30일00:52 경북 상주시 북쪽 20km 지역 규모3.2 지진발생...,cbs_kma
17604,2019/12/30 00:32:26,"53,74,104,119,162","경상남도 전체,경상북도 전체,대구광역시 전체,부산광역시 전체,울산광역시 전체",24998,[기상청] 12월30일00:32 경남 밀양시 동북동쪽 16km 지역 규모3.5 지진...,cbs_kma
17689,2019/10/27 15:40:10,"53,74,104,119,162","경상남도 전체,경상북도 전체,대구광역시 전체,부산광역시 전체,울산광역시 전체",24912,[기상청] 10월27일15:37 경남 창녕군 남쪽 15km 지역 규모3.4 지진발생...,cbs_kma
18083,2019/07/21 11:05:07,"74,104,238","경상북도 전체,대구광역시 전체,충청북도 전체",24517,[기상청] 07월21일11:04 경북 상주시 북북서쪽 11km 지역 규모3.9 지진...,cbs_kma
18190,2019/04/22 05:46:11,"74,104","경상북도 전체,대구광역시 전체",24409,[기상청] 04월22일05:45 경북 울진군 동남동쪽 43km 해역 규모4.0 지진...,cbs_kma
18427,2019/02/10 12:54:35,"74,104","경상북도 전체,대구광역시 전체",24172,[기상청]02월10일12:53 경북 포항시 북구 동북동쪽 58km 해역 규모4.0 ...,cbs_kma


In [10]:
msg_api_df[msg_api_df["send_platform"] == 'SafeOn']

,create_date,location_id,location_name,md101_sn,msg,send_platform
20216,2017/01/27 09:50:04,3100000000,울산광역시,2336,"〔울산광역재난안전대책본부〕안전안내, 설날 당일 옥동공원묘지 일대 교통혼잡 및 교통사...",SafeOn
20227,2017/01/20 14:48:39,4711000000,경상북도 포항시,2325,"[포항시재난안전대책본부]안전안내, 금일 15:00 동해남부앞바다 풍랑경보, 포항시 ...",SafeOn
20263,2016/10/15 20:20:04,3100000000,울산광역시,2289,2016.10.16(일) 예상강수량이 20~60mm로 예상되고 대암댐 자연월류량이 ...,SafeOn
20264,2016/10/15 12:31:51,3100000000,울산광역시,2288,○ 남해안 동부지역 17~18일 대조기로 해수면이 1.5~2.9m 상승되니 저지대 ...,SafeOn
20265,2016/10/15 12:21:23,3100000000,울산광역시,2287,○ 남해안 동부지역 17~18일 대조기로 해수면이 1.5~2.9m 상승되니 저지대 ...,SafeOn
...,...,...,...,...,...,...
22072,2012/01/12 07:10:53,4217000000,강원도 동해시,129,"[풍랑주의보해제] 01.12, 04:00 강원남부앞바다, 강원중부앞바다, 동해중부먼...",SafeOn
22073,2012/01/11 07:29:48,4217000000,강원도 동해시,128,풍랑주의보 발표 01.11. 09:00 발효 동해중부앞바다 사전대비 및 재난예방 철...,SafeOn
22074,2012/01/10 23:27:55,4217000000,강원도 동해시,127,풍랑주의보 발표 동해남부전해상 01월11일 03:00 발효 사전대비 및 재난예방 철...,SafeOn
22075,2012/01/10 22:44:01,4217000000,강원도 동해시,126,풍랑주의보 발표 동해중부먼바다 01월 11일 01:00 발효 사전대비 바랍니다. -...,SafeOn


In [11]:
# 총 379개 행의 'send_platform'열이 'cbs'가 아닌 다른 값을 가짐을 확인
# 분석에 필요 없다고 판단되어 'send_platform'열을 제거

In [15]:
msg_api_df = msg_api_df.drop('send_platform', axis=1)

In [16]:
msg_api_df

,create_date,location_id,location_name,md101_sn,msg
0,2020/08/06 14:20:00,49,경기도 평택시,42609,"[평택시청]관내 하천 상류지역에 내린 집중호우로 위험 할 수 있으니 하천변, 하천둔..."
1,2020/08/06 14:06:06,52,경기도 화성시,42608,[화성시청]코로나19 수원시 113번 확진자(화성시 능동 거주) 발생. 격리병원 이...
2,2020/08/06 14:01:35,21,경기도 전체,42607,[경기도청] 경기도내 산사태 위험수준이 “심각단계”로 산림 인근지역 방문을 자제하시...
3,2020/08/06 13:51:38,"11,20,41","강원도 원주시,강원도 횡성군,경기도 여주시",42606,"[한강홍수통제소]오늘 13:40 섬강 원주시(문막교) 홍수주의보 발령, 안전에 유의..."
4,2020/08/06 12:47:48,8,강원도 양구군,42605,"[양구군청]집중호우에 따라 양구읍 상용터널 내 조명고장에 현재 조사 중으로, 시간이..."
...,...,...,...,...,...
22095,2011/11/20 12:10:41,"3,4,5,6,7,9","강원도 강릉시,강원도 고성군,강원도 동해시,강원도 삼척시,강원도 속초시,강원도 양양군",10,"[국민안전처]20일 12시 동해중부먼바다 풍랑경보, 어선은 출항을 금지하고 출어선은..."
22096,2011/11/18 21:12:20,200,전라남도 해남군,9,"[국민안전처]\n18일21시10분 전남 해남지역 호우경보, 상습침수및위험지역 대피,..."
22097,2011/11/18 11:38:36,217,제주특별자치도 전체,8,"[국민안전처]\n18일11시30분 제주동부지역 호우경보, 상습침수및위험지역 대피,외..."
22098,2011/11/18 08:20:05,217,제주특별자치도 전체,7,"[국민안전처]\n18일8시20분 제주산간,북부지역 호우경보, 상습침수및위험지역 대피..."


In [9]:
msg_api_df["msg"].str.contains("호우") == True

0        False
1         True
2         True
3        False
4        False
         ...  
22195    False
22196    False
22197    False
22198    False
22199    False
Name: msg, Length: 22200, dtype: bool